In [5]:
import geopandas as gpd
import pandas as pd
import matplotlib as plot
import matplotlib.pyplot as plt
import os
from shapely.geometry import Point
import numpy as np
from osgeo import gdal, os, ogr, osr
from shapely.geometry import Polygon
import matplotlib.colors as colors
import pysheds
from pysheds.grid import Grid
import fiona
from fiona.crs import from_epsg
from fiona.crs import from_string



C:\Users\RIPHAWK\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\RIPHAWK\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


### Delineating catchments for each flowtracker measurement


In [9]:
def batch_catchment_delineation(burneddem, pointslist):
    pointslist = pd.read_csv(pointslist)
    for index, row in pointslist.iterrows():
         # make a grid the size of dem
        grid = Grid.from_raster(burneddem, data_name='dem')
        # give grid dem data
        #dem = grid.read_raster(r'D:\Ben_wd\THESIS\watershed\tif\WFK_demsubset4326_fillburn.tif')
        #dem = grid.read_raster(r'D:\Ben_wd\THESIS\watershed\tif\WFK_demsubset4326_fillburn.tif')

        dem = grid.read_raster(burneddem)

        # hydrocondition dem
        pit_filled_dem = grid.fill_pits(dem)
        flooded_dem = grid.fill_depressions(pit_filled_dem)
        inflated_dem = grid.resolve_flats(flooded_dem)
        # # Compute flow direction
        dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
        fdir = grid.flowdir(inflated_dem, dirmap=dirmap)
        # Calculate flow accumulation
        acc = grid.accumulation(fdir, dirmap=dirmap)
        # Delineate a catchment
        #pointslist = pd.read_csv(pointslist)
        # read point xy
        x, y =     row['LONGITUDE'], row['LATITUDE']
        name = str(row['NODE_ID'])
        # Snap pour point 
        #x_snap, y_snap = grid.snap_to_mask(acc > 1000, (x, y))
        x_snap, y_snap = grid.snap_to_mask(acc > 1000, (x, y))

        # Delineate the catchment
        catch = grid.catchment(x=x_snap, y=y_snap, fdir=fdir, dirmap=dirmap, 
                               xytype='coordinate')

        # Crop and export catchment as shapefile the catchment
        grid.clip_to(catch)
        catch_view = grid.view(catch, dtype=np.uint8)
        shapes = grid.polygonize(catch_view)

        # Specify schema
        schema = {'geometry': 'Polygon',
                'properties': {'LABEL': 'float:16'}}
        # Write shapefile
        # Define the Proj4 string for EPSG 4326 (WGS 84)
        crs_proj4 = '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs'

        # Create a CRS object from the Proj4 string
        crs = from_string(crs_proj4)

        
        #with fiona.open(r'D:\Ben_wd\THESIS\watershed\shp\wshdoutput\wgs84\autocatchment4326_'+ str(name) + '.shp', 'w',
        with fiona.open(r'D:\Ben_wd\THESIS\heatsource\WestFork\accretion_model\shp\Hayes10m\autocatchment4326_'+ str(name) + '.shp', 'w',
              
                        driver='ESRI Shapefile',
                        crs=crs,
                        schema=schema) as c:
            i = 0
            for shape, value in shapes:
                rec = {}
                rec['geometry'] = shape
                rec['properties'] = {'LABEL' : str(value)}
                rec['id'] = str(i)
                c.write(rec)
                i += 1








In [10]:
batch_catchment_delineation(burneddem = r'D:\Ben_wd\THESIS\watershed\tif\WFK_demsubset4326_fillburn.tif',
                                     pointslist = r'D:\Ben_wd\THESIS\heatsource\WestFork\accretion_model\nodes_10mHayes.csv')



### Finding the amount of landform classes in each catchment

In [11]:
import os
import geopandas as gpd
import pandas as pd
from pyproj import CRS
from shapely.geometry import box

# Specify the folder containing the larger shapefiles
larger_shapefile_path = r'D:\Ben_wd\THESIS\watershed\shp\WFK_SSURGO\WFK_landformsFINAL6608.shp'

# Specify the folder containing the smaller shapefile for clipping
smaller_shapefiles_folder = r'D:\Ben_wd\THESIS\heatsource\WestFork\accretion_model\shp\Hayes10m'

# Specify the target CRS (EPSG 6608)
target_crs = CRS.from_epsg(6608)

# Read the larger shapefile
larger_gdf = gpd.read_file(larger_shapefile_path)

# Reproject the larger shapefile to the target CRS
larger_gdf = larger_gdf.to_crs(target_crs)

# Create an empty DataFrame to store the results
#results_df = pd.DataFrame(columns=['SmallerShapefile'])
results_df = []
# Loop through smaller shapefiles in the folder
for filename in os.listdir(smaller_shapefiles_folder):
    if filename.endswith(".shp"):
        # Construct the full path to the smaller shapefile
        smaller_shapefile_path = os.path.join(smaller_shapefiles_folder, filename)

        # Read the smaller shapefile
        smaller_gdf = gpd.read_file(smaller_shapefile_path)

        # Reproject the smaller shapefile to the target CRS
        smaller_gdf = smaller_gdf.to_crs(target_crs)

        # Clip the larger shapefile with the smaller shapefile
        clipped_gdf = gpd.overlay(larger_gdf, smaller_gdf, how='intersection')

        # Get a list of unique classes from the clipped result
        unique_classes = clipped_gdf['landform_2'].unique()

        # Create a dictionary to store class areas
        class_areas = {}

        # Calculate areas for each class within the clipped areas
        for class_name in unique_classes:
            class_gdf = clipped_gdf[clipped_gdf['landform_2'] == class_name]
            class_area = class_gdf.geometry.area.sum()/1000000
            class_areas[class_name] = class_area

        # Add results to the DataFrame
        #results_df = results_df.append({'SmallerShapefile': filename, **class_areas})#, ignore_index=True)
        result_dict = {'SmallerShapefile': filename, **class_areas}
        results_df.append(result_dict)

# Fill any NaN values in the DataFrame with 0
#results_df = results_df.fillna(0)
selected_rows = results_df.iloc[2::3]
selected_rows['TotalArea'] = results_df.sum(axis=1)
#selected_rows.rename(columns={'SmallerShapefile': 'Site', '1' : 'Valley', '2': 'Hillside', '3': 'Ridgetop'}, inplace=True)

# Save the results DataFrame to a CSV file
selected_rows.to_csv(r'D:\Ben_wd\THESIS\heatsource\WestFork\accretion_model\accretion_Hayes10m.csv', index=False)

print('Class areas within the clipped areas have been calculated and saved to accretion.csv')


AttributeError: 'list' object has no attribute 'fillna'

In [14]:
results_df = pd.concat(results_df, ignore_index=True)


TypeError: cannot concatenate object of type '<class 'dict'>'; only Series and DataFrame objs are valid

In [13]:
selected_rows = results_df.iloc[2::3]
selected_rows['TotalArea'] = results_df.sum(axis=1)

AttributeError: 'list' object has no attribute 'iloc'

In [15]:
import os
import geopandas as gpd
import pandas as pd
from pyproj import CRS
from shapely.geometry import box

# Specify the folder containing the larger shapefiles
larger_shapefile_path = r'D:\Ben_wd\THESIS\watershed\shp\WFK_SSURGO\WFK_landformsFINAL6608.shp'

# Specify the folder containing the smaller shapefile for clipping
smaller_shapefiles_folder = r'D:\Ben_wd\THESIS\heatsource\WestFork\accretion_model\shp\Hayes10m'

# Specify the target CRS (EPSG 6608)
target_crs = CRS.from_epsg(6608)

# Read the larger shapefile
larger_gdf = gpd.read_file(larger_shapefile_path)

# Reproject the larger shapefile to the target CRS
larger_gdf = larger_gdf.to_crs(target_crs)
# Create an empty DataFrame to store the results
dfs_list = []

# Loop through smaller shapefiles in the folder
for filename in os.listdir(smaller_shapefiles_folder):
    if filename.endswith(".shp"):
        # Construct the full path to the smaller shapefile
        smaller_shapefile_path = os.path.join(smaller_shapefiles_folder, filename)

        # Read the smaller shapefile
        smaller_gdf = gpd.read_file(smaller_shapefile_path)

        # Reproject the smaller shapefile to the target CRS
        smaller_gdf = smaller_gdf.to_crs(target_crs)

        # Clip the larger shapefile with the smaller shapefile
        clipped_gdf = gpd.overlay(larger_gdf, smaller_gdf, how='intersection')

        # Get a list of unique classes from the clipped result
        unique_classes = clipped_gdf['landform_2'].unique()

        # Create a dictionary to store class areas
        class_areas = {}

        # Calculate areas for each class within the clipped areas
        for class_name in unique_classes:
            class_gdf = clipped_gdf[clipped_gdf['landform_2'] == class_name]
            class_area = class_gdf.geometry.area.sum() / 1000000
            class_areas[class_name] = class_area

        # Add results to the DataFrame
        result_df = pd.DataFrame([{'SmallerShapefile': filename, **class_areas}])

        # Add the DataFrame to the list
        dfs_list.append(result_df)
        
results_df['TotalArea'] = results_df.iloc[:, -3:].sum(axis=1)
results_df = results_df.sort_values(by='TotalArea', ascending=False)


# Save the results DataFrame to a CSV file
results_df.to_csv(r'D:\Ben_wd\THESIS\heatsource\WestFork\accretion_model\accretion_Hayes10m.csv', index=False)

print('Class areas within the clipped areas have been calculated and saved to accretion.csv')


AttributeError: 'list' object has no attribute 'iloc'

In [21]:
#results_df = pd.DataFrame(results_df)
#results_df['TotalArea'] = results_df.iloc[:, -3:].sum(axis=1)
#results_df = results_df.sort_values(by='TotalArea', ascending=False)
results_df.to_csv(r'D:\Ben_wd\THESIS\heatsource\WestFork\accretion_model\accretion_Hayes10m.csv', index=False)

results_df

,SmallerShapefile,1,2,3,TotalArea
0,autocatchment4326_0.0.shp,16.710253,33.896155,128.153225,178.759632
1,autocatchment4326_1.0.shp,16.709946,33.896155,128.153225,178.759325
1059,autocatchment4326_3.0.shp,16.697709,33.896155,128.153225,178.747088
948,autocatchment4326_2.0.shp,16.697709,33.896155,128.153225,178.747088
1170,autocatchment4326_4.0.shp,16.697171,33.896155,128.153225,178.746550
...,...,...,...,...,...
723,autocatchment4326_1649.0.shp,0.062266,0.223297,0.387687,0.673250
1299,autocatchment4326_515.0.shp,0.000254,0.153690,0.166241,0.320185
1700,autocatchment4326_877.0.shp,0.168345,0.084068,0.054322,0.306735
1507,autocatchment4326_702.0.shp,0.007797,0.059670,0.124207,0.191674


### Joining Discharge data and Landform proportions for baseflow estimate calculations

In [22]:
df = pd.read_csv(r'D:\Ben_wd\THESIS\heatsource\WestFork\accretion_model\accretion_Hayes10m.csv')
df.rename(columns={'SmallerShapefile': 'Site', '1' : 'Valley', '2': 'Hillside', '3': 'Ridgetop'}, inplace=True)
#sites =pd.read_csv(r'D:\Ben_wd\THESIS\flowtracker\FlowtrackerXYDischarge_nooutliers.csv')

In [29]:
df = pd.read_csv(r'D:\Ben_wd\THESIS\heatsource\WestFork\accretion_model\accretion_Hayes10m2.csv')

df["baseflow_juckem"] = (df["Valley"]*20.1*.0003169)+(df["Hillside"]*36.3*.0003)+(df["Ridgetop"]*16*.0003)
df_sorted_by_area = df.sort_values(by='TotalArea')

#df_sorted_by_area.to_csv(r'D:\Ben_wd\THESIS\heatsource\WestFork\accretion_model\accretion_Hayes10m2.csv')
df

,Unnamed: 0,Site,Valley,Hillside,Ridgetop,TotalArea,baseflow_juckem
0,1804,autocatchment4326_1835.0.shp,3.664337,6.382909,42.480806,52.528052,0.296758
1,1803,autocatchment4326_1834.0.shp,3.664600,6.383568,42.480806,52.528974,0.296767
2,1801,autocatchment4326_1833.0.shp,3.693166,6.414944,42.502374,52.610484,0.297394
3,1802,autocatchment4326_1832.0.shp,3.693166,6.414944,42.502374,52.610484,0.297394
4,1800,autocatchment4326_1831.0.shp,3.693851,6.418318,42.504307,52.616476,0.297445
...,...,...,...,...,...,...,...
1800,4,autocatchment4326_4.0.shp,16.697171,33.896155,128.153224,178.746550,1.090620
1801,3,autocatchment4326_2.0.shp,16.697709,33.896155,128.153224,178.747088,1.090624
1802,2,autocatchment4326_3.0.shp,16.697709,33.896155,128.153224,178.747088,1.090624
1803,1,autocatchment4326_1.0.shp,16.709946,33.896155,128.153224,178.759325,1.090702


In [27]:
df

,Unnamed: 0,Site,Valley,Hillside,Ridgetop,TotalArea,baseflow_juckem
0,1835,autocatchment4326_703.0.shp,0.007797,0.059670,0.124207,0.191674,0.001296
1,1834,autocatchment4326_702.0.shp,0.007797,0.059670,0.124207,0.191674,0.001296
2,1833,autocatchment4326_877.0.shp,0.168345,0.084068,0.054322,0.306735,0.002249
3,1832,autocatchment4326_515.0.shp,0.000254,0.153690,0.166241,0.320185,0.002473
4,1831,autocatchment4326_1649.0.shp,0.062266,0.223297,0.387687,0.673250,0.004689
...,...,...,...,...,...,...,...
1831,4,autocatchment4326_4.0.shp,16.697171,33.896155,128.153225,178.746550,1.090620
1832,3,autocatchment4326_2.0.shp,16.697709,33.896155,128.153225,178.747088,1.090624
1833,2,autocatchment4326_3.0.shp,16.697709,33.896155,128.153225,178.747088,1.090624
1834,1,autocatchment4326_1.0.shp,16.709946,33.896155,128.153225,178.759325,1.090702


In [54]:
df_sorted_by_area['accretion'] = df_sorted_by_area['baseflow_juckem'].diff()
df_sorted_by_area.to_csv(r'D:\Ben_wd\THESIS\heatsource\WestFork\accretion_model\HSmodelvalues_Hayes10m2.csv')



In [53]:
df_sorted_by_area

,Site,Valley,Hillside,Ridgetop,TotalArea,baseflow_juckem
192,autocatchment4326_192.0.shp,1.650016,3.063865,24.926670,29.640550,0.163524
191,autocatchment4326_191.0.shp,1.658472,3.079470,24.928266,29.666207,0.163755
190,autocatchment4326_190.0.shp,1.674685,3.105806,24.931729,29.712220,0.164162
189,autocatchment4326_189.0.shp,1.688207,3.119324,24.931729,29.739261,0.164395
188,autocatchment4326_188.0.shp,1.696471,3.217637,25.192735,30.106842,0.166771
...,...,...,...,...,...,...
4,autocatchment4326_4.0.shp,16.612905,33.741177,128.025866,178.379948,1.087785
3,autocatchment4326_3.0.shp,16.667819,33.772396,128.033847,178.474062,1.088513
2,autocatchment4326_2.0.shp,16.687347,33.887970,128.153225,178.728542,1.090469
1,autocatchment4326_1.0.shp,16.694246,33.896155,128.153225,178.743625,1.090602
